# __Lecture 09: Practical analyses in Python (continued)__

----
#### __Announcement:__ Homework 4 will be due on Nov 2

#### __On Tuesday, we talked about:__
- how to define functions in python
- the `re` module

#### __Today we will:__
- practice using the `re` module
- learn about (and practice) using the `biopython` module

###  
----

## __Review:__ Regular expressions

The `re` module offers a set of functions that allows us to search a string for a match using a __search pattern__ 

Here are some _common elements_ to have in your search pattern:
* __letter characters__ which returns a match where the string contains the specified letter (e.g. `A`, `B`, `C`, ...)
* __special characters__ which returns a match where the string contains the specified special character; these must be preceded by a `\`
* `\d` which returns a match where the string contains __digits__ (numbers from `0`-`9`)

You may also want to add the following _customizations_:
* `.` matches __any single character__
* `[]` specifies a __set of characters__ to search for (e.g. `[a-n]`)
* `()` __capture and group__ everything contained inside, and search the string for everything together
* `?P<name>` indicates a __named search pattern__ group with name `name`
* `+` specifies __one or more occurrences__ of a certain pattern element
* `{}` specifies __exactly the specified number of occurrences__ of a certain pattern element
* `$` specifies the __end of the string__
* `^` specifies the __beginning of the string__

Always remember to import the `re` module if you'd like to work with regular expressions:

In [1]:
import re

## __Review practice:__ using regular expressions to parse barcodes
Now we will use regular expressions to parse barcodes from nucleotide sequences.
For instance, you might have to do this in a single-cell RNA-seq experiment where there is a barcode at the end of each read telling you the cell that the read came from.

Imagine that our valid molecules should have sequences like this:

`CTAGCNNNNNNGATCA`

See how there is a 6-nucleotide barcode in the center of the sequence.
We have a list of sequences, and want to parse through them to figure out which ones meet the expected pattern--and get the barcode from those that do:

In [2]:
seqs = ['CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CCAGCatagcaGATCA',  # does not have expected 5' sequence
        'CTAGCtacagGATCA',   # barcode too short
        'CTAGCgaccatGATCA',  # has barcode GACCAT
        'CTAGCatcgatGATCA',  # has barcode ATCGAT
        'CTAGCatcgatGGTCA',  # does not have expected 3' sequence
        ]

Write a function that parses these barcoded sequences and gets the ones with valid barcodes.
In doing this, note that:

  1. If you have a string `s`, `s.upper()` makes it all uppercase.
  2. You may want to start searching at the beginning of the string (using the `^` symbol in the search pattern)
    
Below I've written the function documentation, try to implement it.
__Take a few minutes in groups to work through this.__

In [72]:
def count_barcodes(seqs, bclen=6, upstream='CTAGC', downstream='GATCA'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs : list
        DNA sequences.
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
    downstream : str
        Sequence downstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    barcode_id = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})"+downstream) #bclen is itg so need to conver it to string
    barcodes = {}

    for seq in seqs:
        seq = seq.upper()
        match = barcode_id.search(seq)
        print(match)
        if match: # checks to see if there is a valid match
            b = match.group("barcode")
            if b in barcodes: # checks to see if b is a key in the dictionary
                barcodes[b] += 1
            else:
                barcodes[b] = 1

    return barcodes

count_barcodes(seqs)

<re.Match object; span=(0, 16), match='CTAGCATCGATGATCA'>
None
None
<re.Match object; span=(0, 16), match='CTAGCGACCATGATCA'>
<re.Match object; span=(0, 16), match='CTAGCATCGATGATCA'>
None


{'ATCGAT': 2, 'GACCAT': 1}

In [1]:
def count_barcodes(seqs, bclen=6, upstream='CTAGC', downstream='GATCA'):
    """Parse and count barcodes.
    
    Parameters
    ----------
    seqs : list
        DNA sequences.
    bclen : int
        Length of barcode
    upstream : str
        Sequence upstream of barcode.
    downstream : str
        Sequence downstream of barcode.
        
    Returns
    -------
    dict
        Keyed by each valid barcode, value is number of times the barcode
        is observed.
        
    Note
    ----
    The function is **not** case-sensitive, and all barcodes are reported
    in upper-case.
    
    """
    
    
    
    barcode_id = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})" + downstream)
    for seq in seqs:
        seq = seq.upper()
        match = barcode_id.search(seq)
        print (match)
        return barcode_id

Run the function once you've implemented it. Does it give the right result?

In [31]:
count_barcodes(seqs)

{'ATCGAT': 2, 'GACCAT': 1}

## __Biopython__
[Biopython](https://biopython.org/) is a package that has lots of useful functions for computational biology.

It is very handy for things like __reading in sequences__ in many different formats and __processing sequence data__: the subpackage [Bio.SeqIO](https://biopython.org/wiki/SeqIO) is your friend!

_(Do note that if you are analyzing truly large datasets, `Biopython` is not very fast and you may want to use something like [pysam](https://pysam.readthedocs.io/en/latest/api.html); but `Biopython` is a good starting point)._

### Reading in a file
I have included the file [barcodes_R1.fastq](barcodes_R1.fastq), which has some FASTQ sequences in it.

Let's use `Biopython` to read the FASTQ entries.

First, we'll need to import `Biopython.SeqIO`

In [2]:
import Bio.SeqIO

Now read in the sequencing reads and convert them to a list:

In [4]:
reads = Bio.SeqIO.parse('barcodes_R1.fastq', format='fastq')
seqreads = list(reads)

How many reads were there?

In [5]:
len(seqreads)

10000

In [6]:
print(f"Found {len(seqreads)} sequencing reads.")

Found 10000 sequencing reads.


Let's look at the first read:

In [9]:
seqreads[0]

SeqRecord(seq=Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'), id='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', name='HISEQ:621:HMJGNBCX2:1:1101:1797:2150', description='HISEQ:621:HMJGNBCX2:1:1101:1797:2150 1:N:0:ATGTCA', dbxrefs=[])

You can see that `BioPython` reads the sequences in as [SeqRecord](https://biopython.org/wiki/SeqRecord) objects, which have a lot of information, including the header, quality scores, etc.

If you want to just access the sequence element of each `SeqRecord`, you can do this as follows:

In [33]:
seqreads[0].seq

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

Let's make a list of sequences from our `seqreads`

In [36]:
seqreads_Seq = []
for seqrecord in seqreads:
    sequence = seqrecord.seq # isolate the sequence from the seqrecord
    seqreads_Seq.append(sequence) # add string sequence to list
seqreads_Seq[0]

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

In [15]:
seqreads_Seq[0:5]

[Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA'),
 Seq('CTTCCTGGTCACGGTTGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CGA'),
 Seq('CACGGCTATTACCCCGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAA...CGA'),
 Seq('TCAGCGATGAATGTGGGCGGCCGCCTATGTTGCATCATTATATGCAAATCCGGC...TGA'),
 Seq('ATATGGGAGACGATAAGCGGCCGCCTATGGTGCATCATTATATGCAAATCCGGC...CTC')]

These sequences are a `Seq` objects

In [17]:
type(seqreads_Seq[0])

Bio.Seq.Seq

If you want to convert the sequences to strings, you can do so as follows:

In [9]:
str(seqreads_Seq[0])

'GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA'

Likewise, strings are not `Seq` objects. If you would like to create a `Seq` object from a string, you can do so as follows:

In [10]:
# This requires us to first import the Seq class
from Bio.Seq import Seq

Seq('ATTGCT')

Seq('ATTGCT')

### Built-in methods

`Biopython` has many useful built-in functions for working with sequencing data. 
We will discuss a few examples in class from the submodule [Bio.Seq](https://biopython.org/docs/1.75/api/Bio.Seq.html), but feel free to read about more [here](https://biopython.org/wiki/Seq)

In [18]:
seq = seqreads_Seq[0]
seq

Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA')

We can use this module to get the __complement__ and __reverse complement__ of a sequence:

In [269]:
seq.complement()

Seq('CGAATTCAATAAATCACGCCGGCGGATACCACGTGATAATAAATAGATAGCACT...TCT')

In [271]:
seq.reverse_complement()

Seq('TCTGCTATTCCGTGTCAATAAACATCGTCGTTGCATGCCTTCATTTTAGTGCTG...AGC')

We can use this module to __transcribe__ and __translate__ a sequence:

In [14]:
seq.transcribe()

Seq('GCUUAAGUUAUUUAGUGCGGCCGCCUAUGGUGCACUAUUAUUUAUCUAUCGUGA...AGA')

In [16]:
seq.translate()

Seq('A*VI*CGRLWCTIIYLS*KGVLLHQAKIGRAHV*TPVTCQNLVCRLLLEKKKKA...E*Q')

__If you choose to use these methods, remember that a `Seq` object is not a string. You will need to convert your sequence back to a string before using methods/functions that require strings.__

## __A real biological analysis: parsing barcodes__
<a id='real_analysis'></a>
The reads that we just read as `seqreads_Seq` come from a real sequencing run of influenza virus HA and NA genes.

The __actual sequences__ are as follows:

    5'-[end of HA/NA]-AGGCGGCCGC-[16 X N barcode]-3'

    
The __sequencing run reads__ from the reverse end of the molecules (in 5'>3' orientation), so the sequencing reads are as follows:

    5'-[reverse complement of 16 X N barcode]-GCGGCCGCCT-[reverse complement of the end of HA/NA]-3'




We want to determine which reads have valid sequences, get the barcodes out of strings, and count the barcodes.
So this requires setting up an analysis that does the following:

 1. Get the reverse complement of each read.
 2. Determine if it matches the expected pattern (with the correct length barcode and constant sequence)
 3. If it matches, extract the barcode and add it to a dictionary to keep track of counts.

### __Group activity__
In groups, work together to write some code to do this.
I have created a code chunk for each step (with some parts filled in). 
Remember to run the code chunks in the correct order!

For your homework, you will be asked to extend this in-class analysis to get statistics for HA and NA seperately.

In [3]:
# load necessary packages
import re
import Bio.SeqIO
from Bio.Seq import Seq

__Step 1:__ You'll need to write a function that identifies a barcode with a known upstream sequence. 
I've provided the documentation here--__try writing this function on your own.__

_Hint: we wrote a similar function yesterday_

_Hint 2: You can use the built-in reverse complement method_

_Hint 3: You will want to convert the sequence to a string before searching for regular expressions_

In [3]:
def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):
    """Identify barcode with known upstream sequence.
    
    Parameters
    ----------
    seqread : Seq object
        Nucleotide sequence read matching UPSTREAM-BARCODE in reverse orientation.
    bclen : int
        Length of barcode
    upstream: str
        Sequence upstream of the barcode.
        
    Returns
    -------
    str or None
        Sequence of the barcode in the forward orientation, or `None` if no match to expected barcoded sequence.
        
    Example
    -------
    >>> read_barcode('TTTTTTTTTTTTTTTTGCGGCCGCCT', bclen=16)
    'AAAAAAAAAAAAAAAA'
        
    """
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})$")
    
    match = barcode_re.search(sequence_str)
    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode
    
#read_barcode(Seq('TTTTTTTTTTTTTTTTGCGGCCGCCT'), bclen=16)


In [4]:
def read_barcode(seqread, bclen, upstream='AGGCGGCCGC'):

    # get the reverse complement of the read
    reverse = seqread.reverse_complement(seqread)
    
    
    # compile the barcode search pattern
    barcode_pattern = re.compile(upstream + f"(?P<barcode>[ATCGN]{{{bclen}}})$")
    
    # search for the barcode pattern
    match = barcode_pattern.search(reverse)
    
    if match:
        barcode = match.group('barcode')
        return barcode
    else:
        return None

__Step 2:__ Read sequences from the barcodes_R1.fastq file and create a list of only the sequences (as Seq objects). __We already did this step earlier__

In [5]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)



__Step 3:__ Get the counts of all barcodes. _(Hint: you might want to store barcodes and counts in a dictionary, and also keep track of the number of sequences that don't have a valid barcode)_

Please name your dictionary `barcode_counts`

In [7]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def read_barcode(seqread, bclen=16, upstream='AGGCGGCCGC'):

    
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})$")
    
    match = barcode_re.search(sequence_str)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode


barcode_counts = {}
n_invalid = 0

for seq in seqreads_Seq: # iterate through all reads
    barcode = read_barcode(seq, bclen = 16, upstream='AGGCGGCCGC')
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in barcode_counts:
            barcode_counts[barcode] += 1
        else:
            barcode_counts[barcode] = 1
    else:
        n_invalid += 1

print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")



There were 10000 total sequence reads parsed, and 569 lacked a valid barcode.


In [1]:
# run this code chunk...
seqreads = list(Bio.SeqIO.parse('barcodes_R1.fastq', 'fastq'))

seqreads_Seq = []
for seqrecord in seqreads:
    seqreads_Seq.append(seqrecord.seq)

def read_barcode(seqread, upstream='CACGATAGATAAATAATAGTGCACCAT'):

    
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    
    barcode_re = re.compile(upstream + "(?P<barcode>[ATCG])")
    
    match = barcode_re.search(sequence_str)

    if match:
        barcode = match.group("barcode")
    else:
        barcode = None
    
    return barcode


barcode_counts = {}
n_invalid = 0

for seq in seqreads_Seq: # iterate through all reads
    barcode = read_barcode(seq, upstream='CACGATAGATAAATAATAGTGCACCAT')
    if barcode: # if there is a valid barcode, add it to the dictionary
        if barcode in barcode_counts:
            barcode_counts[barcode] += 1
        else:
            barcode_counts[barcode] = 1
    else:
        n_invalid += 1

print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")



NameError: name 'Bio' is not defined

In [ ]:

print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")


In [ ]:
def extract_barcode_and_gene(read):
    # Define the patterns for HA and NA sequences

    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)

    ha_pattern = re.compile(r'CCGGATTTGCATATAATGATGCACCAT')  # Pattern for HA gene
    na_pattern = re.compile(r'CACGATAGATAAATAATAGTGCACCAT')  # Pattern for NA gene

    # Get the reverse complement of the read
    reverse_read = Seq(str(read).reverse_complement())
    

    # Search for the gene patterns in the reverse complement
    ha_match = ha_pattern.search(reverse_read)
    na_match = na_pattern.search(reverse_read)

__Step 4:__ Report the total number of sequences parsed, and how many lacked a valid barcode.

In [337]:
# your code here...
print(f"There were {len(seqreads_Seq)} total sequence reads parsed, and {n_invalid} lacked a valid barcode.")


There were 10000 total sequence reads parsed, and 569 lacked a valid barcode.


In [ ]:
# Define the function to extract barcode and identify the gene
def extract_barcode_and_gene(seqread):
    # Define the patterns for HA and NA sequences
    ha_pattern = re.compile(r'CCGGATTTGCATATAATGATGCACCAT')  # Pattern for HA gene
    na_pattern = re.compile(r'CACGATAGATAAATAATAGTGCACCAT')  # Pattern for NA gene

    # Get the reverse complement of the read
    sequence = seqread.reverse_complement()
    sequence_str = str(sequence)
    

    # Search for the gene patterns in the reverse complement
    ha_match = ha_pattern.search(sequence_str)
    na_match = na_pattern.search(sequence_str)

    # Check if the read matches HA or NA gene, extract barcode, and count
    if ha_match:
        barcode = extract_barcode(sequence_str, ha_match.end())
        return 'HA', barcode
    elif na_match:
        barcode = extract_barcode(sequence_str, na_match.end())
        return 'NA', barcode
    else:
        return None, None  # Return None if neither HA nor NA gene is found
    

In [18]:
# Define a function to extract the barcode
def extract_barcode(seqread, start):
    barcode_length = 16
    barcode = read[start - barcode_length: start]  # Extract the barcode
    return barcode

# Create dictionaries to store barcode counts for HA and NA
ha_barcodes = {}
na_barcodes = {}

In [19]:
# Iterate through the FASTQ file or sequence reads
for seq in seqreads_Seq:
    gene, barcode = extract_barcode_and_gene(seq)
    if gene == 'HA':
        if barcode in ha_barcodes:
            ha_barcodes[barcode] += 1
        else:
            ha_barcodes[barcode] = 1
    elif gene == 'NA':
        if barcode in na_barcodes:
            na_barcodes[barcode] += 1
        else:
            na_barcodes[barcode] = 1

# Display barcode counts for HA and NA genes
print("Barcode counts for HA gene:")
print(ha_barcodes)

print("\nBarcode counts for NA gene:")
print(na_barcodes)

NameError: name 'read' is not defined

In [2]:
def count_barcodes(seqs, bclen=6, upstream='CTAGC', downstream='GATCA'):

    barcode_id = re.compile(upstream + "(?P<barcode>[ATCG]{" + str(bclen) + "})"+downstream) #bclen is itg so need to conver it to string
    barcodes = {}

    for seq in seqs:
        seq = seq.upper()
        match = barcode_id.search(seq)
        print(match)
        if match: # checks to see if there is a valid match
            b = match.group("barcode")
            if b in barcodes: # checks to see if b is a key in the dictionary
                barcodes[b] += 1
            else:
                barcodes[b] = 1

    return barcodes

count_barcodes(seqs)

NameError: name 'seqs' is not defined